In [1]:
# Import the necessary modules
import os
import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm
from dataset import MyDataset
from model import MaturityPrediction

In [2]:
# Initialize the maturity prediction model:
model = MaturityPrediction()

# Define the location of the saved weigths:
model_path = '../data/model_multi_UIUC.pth'

# Load the weigths of the trained model:
model.load_state_dict(torch.load('../data/model_2018.pth'))

# Set the model in evaluation mode:
model.eval()

print('Model OK')

Model OK


In [3]:
main_dir = '../examples/'
field_codes = ['2018_F01A', '2018_F02A', '2018_F02B', '2019_F03A', '2019_F04A', '2019_F05A', '2020_F06A', '2020_F06B']

In [4]:
# Loop trough all fields:
for field_code in field_codes:

    # Directory with the images:
    img_dir = f'../../../GDM_2019/data/paper/{field_code}'
    
    # Create a dataset from the directory path:
    test_dataset =  MyDataset(img_dir)

    # Create an empty list to store the results:
    preds_list = []
    
    # Get each observation in the dataset:
    for Plot_ID, images, img_dates in tqdm(test_dataset):
        
        # No need to record gradients in prediction
        with torch.no_grad():
            
            # Make the prediction for this set of images:
            pred = model(images[None], img_dates)
        
        # Round the prediction with one decimal place:
        pred = np.round(pred.data.item(), 1)
        
        # Append the ID and the prediciton to the list:
        preds_list.append({'Plot_ID':Plot_ID, 'Mat':pred})

    # Create a DataFrame from the list:
    df_pred = pd.DataFrame(preds_list)
    
    # Export the results to a CSV file:
    df_pred.to_csv(f'{img_dir}.csv', index = False)
